In [4]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/Colab Notebooks/TheraBot-main

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/TheraBot-main


In [5]:
import keras
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format = 'retina'
import itertools, pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

classes = ["neutral", "happy", "sad", "love", "anger"]

In [6]:
model = load_model("therabot.h5")

In [7]:
emotion_scores = {
    "neutral" : 0,
    "happy" : 0,
    "sad" : 0,
    "love" : 0,
    "anger" : 0
}

def emotion_score(y_prob):
    emotion_scores["neutral"] += y_prob[0][0]
    emotion_scores["happy"] += y_prob[0][1]
    emotion_scores["sad"] += y_prob[0][2]
    emotion_scores["love"] += y_prob[0][3]
    emotion_scores["anger"] += y_prob[0][4]
    
def get_highest_key():
  highest_value = 0
  for key in emotion_scores.keys():
    if emotion_scores[key] >= highest_value:
      highest_value = emotion_scores[key]
      highest_key = key
  return highest_key

def consolidation_message(highest_key):
    if highest_key == 'neutral':
      print("Therabot: There seem to be nothing going on right now. I hope nothing bad comes in your way " + user_name)
    elif highest_key == 'happy':
      print("Therabot: " + user_name + "! You seem to be in a good mood, hope you feel this way always")
    elif highest_key == 'sad':
      print("Therabot: " + user_name + ", you seem to be really sad, maybe consult a therapist or talk to a close friend")
    elif highest_key == 'love':
      print("Therabot: I am happy that you, " + user_name + "is deeply infatuated. I hope that your love stays blooming forever!")
    elif highest_key == 'anger':
        print("Therabot: You seem to be really agitated right now. Take a deep breath and release your tension " + user_name + ". Everything will be fine as time goes")
    else:
      print("Goodbye " + user_name + "! Take care!")
    
def reply(detected_intent):
    if detected_intent == 'neutral':
      print("Therabot: Neutral detected")
    elif detected_intent == 'happy':
      print("Therabot: Happy detected!")
    elif detected_intent == 'sad':
      print("Therabot: Sad detected")
    elif detected_intent == 'love':
      print("Therabot: Love detected")
    elif detected_intent == 'anger':
        print("Therabot: Anger detected")
    
def fallback_intent():
    print("Sorry I don't understand. Can you elaborate please?")
    
def analyze_message(user_message):
    text = [user_message]
    sequences_test = tokenizer.texts_to_sequences(text)
    MAX_SEQUENCE_LENGTH = 30
    data_int_t = pad_sequences(sequences_test, padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
    data_test = pad_sequences(data_int_t, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))
    return data_test

def predict_emotion(data_test, y_prob):
#     y_prob = model.predict(data_test)
    for n, prediction in enumerate(y_prob):
        pred = y_prob.argmax(axis=-1)[n]
#         print(y_prob[0])
    return pred

In [8]:
from keras.preprocessing.sequence import pad_sequences

print("Please enter your name: ")
user_name = input()
print("Therabot: Welcome " + user_name +  ", my name is TheraBot")
print("Therabot: How are you feeling "+ user_name + " ?")
while True:
    print("User: ", end="")
    user_message = input()
    if user_message!="quit":
        data_test = analyze_message(user_message)
        y_prob = model.predict(data_test)
        pred = predict_emotion(data_test,y_prob)
        #print(y_prob[0])
        highest_emotion_confidence = y_prob[0][pred]
        if highest_emotion_confidence > 0.33:
            reply(classes[pred]) 
        else:
            fallback_intent()
        emotion_score(y_prob)
    elif user_message.lower() == "quit":
        highest_key = get_highest_key()
        consolidation_message(highest_key)
        break

Please enter your name: 
Brian
Therabot: Welcome Brian, my name is TheraBot
Therabot: How are you feeling Brian ?
User: I am feeling great
Therabot: Sad detected
User: why u like this
Therabot: Neutral detected
User: haih
Therabot: Neutral detected
User: i am very dissapointed in u chatbot
Therabot: Sad detected
User: quit la diu
Therabot: Neutral detected
User: quit
Therabot: Brian , you seem to be really sad, maybe consult a therapist or talk to a close friend
